Notebook for adjusting the animation scripts to work for two dimensional outputs from inference and adjusting difference code to work for both Norkyst and for two dimensional outputs

In [96]:
#animation of results
from anemoi.datasets import open_dataset
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import cmocean  
import numpy as np 
import sys
import xarray as xr
import cmocean
import cartopy.crs as ccrs
import cartopy

In [78]:
inference_path = '/lustre/storeB/project/fou/hi/foccus/malene/run-anemoi-ocean/ppi/external_checkpoint_inference/Inference_res/2024-04-02_.ckpt_e010_s020000.nc'

df = xr.open_dataset(inference_path)
df

<xarray.Dataset> Size: 5GB
Dimensions:            (X: 1148, Y: 2747, time: 17)
Coordinates:
  * X                  (X) float64 9kB 0.0 800.0 1.6e+03 ... 9.168e+05 9.176e+05
  * Y                  (Y) float64 22kB 0.0 800.0 ... 2.196e+06 2.197e+06
  * time               (time) datetime64[ns] 136B 2024-04-02 ... 2024-04-04
Data variables: (12/26)
    latitude           (X, Y) float32 13MB ...
    longitude          (X, Y) float32 13MB ...
    h                  (time, X, Y) float32 214MB ...
    salinity_0         (time, X, Y) float32 214MB ...
    sea_mask           (time, X, Y) float32 214MB ...
    temperature_0      (time, X, Y) float32 214MB ...
    ...                 ...
    rain               (time, X, Y) float32 214MB ...
    river_binary_mask  (time, X, Y) float32 214MB ...
    sin_julian_day     (time, X, Y) float32 214MB ...
    sin_latitude       (time, X, Y) float32 214MB ...
    sin_local_time     (time, X, Y) float32 214MB ...
    sin_longitude      (time, X, Y) float32 214MB ...

In [97]:
#Making sure the general equations work for the inference files:
def results_animation(file_path,variable, dir, frame, start_time, **kwargs):
    ds = xr.open_dataset(file_path, engine="netcdf4") #add isel when its relevant to select which s-layer you want to look at (per now it is only the surface layer)
    ds_var = ds[f'{variable}']
    print(f'Dataset imported and variable: {variable} is chosen')
    longitude = ds["X"].values#.flatten()
    latitude = ds["Y"].values#.flatten() 
    print(f'Shape of variable is: {ds_var.shape}')
    print(f'shape of longitude is: {longitude.shape}')
    print(f'shape of latitude is: {latitude.shape}')

    fig,ax = plt.subplots(figsize = (12,8), subplot_kw = {'projection' : ccrs.NorthPolarStereo()})
    sc = ax.pcolormesh(ds_var[0], cmap = cmocean.cm.speed, transform = ccrs.PlateCarree(), **kwargs)
    ax.add_feature(cartopy.feature.LAND, zorder = 1, edgecolor = 'black')
    print('Creating a scatterplot is succsessfull')
    cbar = fig.colorbar(sc, ax=ax, orientation = "vertical", label = variable, extend = 'both')
        
    def update(frame):
        print('Made it to the animation loop')
        sc.set_array(ds_var[frame])
        print('Made it to creating the scatter array (ani)')
        ax.set_title(f'Time step: {frame *3} hrs')
        ax.set_xlabel(f'Latitude [$\circ$]')
        ax.set_ylabel(f'Longitude [$\circ$]')
        return sc 
    print('Made it out of the loop')
    ani = FuncAnimation(fig, update, frames=range(frame), interval = 400)
    print('Trying to save it')
    ani.save(f'{dir}/animation_{variable}.gif', writer="imagemagick")


In [ ]:
saving_path = '/lustre/storeB/project/fou/hi/foccus/malene/ocean-ai/plot/tests_animations'
inference_path = '/lustre/storeB/project/fou/hi/foccus/malene/run-anemoi-ocean/ppi/external_checkpoint_inference/Inference_res/2024-04-02_.ckpt_e010_s020000.nc'
variable_inf = 'v_northward_0'
frames = 16 
start = 0 
results_animation(inference_path, variable_inf, saving_path, frames, start)

Dataset imported and variable: v_northward_0 is chosen
Shape of variable is: (17, 1148, 2747)
shape of longitude is: (1148,)
shape of latitude is: (2747,)
Creating a scatterplot is succsessfull
Made it out of the loop
Trying to save it
Made it to the animation loop
Made it to creating the scatter array (ani)
Made it to the animation loop
Made it to creating the scatter array (ani)


/lustre/storeB/project/fou/hi/foccus/.venv/lib64/python3.11/site-packages/cartopy/io/__init__.py:241: DownloadWarning: Downloading: https://naturalearth.s3.amazonaws.com/110m_physical/ne_110m_land.zip
  warnings.warn(f'Downloading: {url}', DownloadWarning)


Made it to the animation loop
Made it to creating the scatter array (ani)
Made it to the animation loop
Made it to creating the scatter array (ani)
Made it to the animation loop
Made it to creating the scatter array (ani)


CalledProcessError: Command '['convert', '-size', '1200x800', '-depth', '8', '-delay', '40.0', '-loop', '0', 'rgba:-', '-layers', 'OptimizePlus', '/lustre/storeB/project/fou/hi/foccus/malene/ocean-ai/plot/tests_animations/animation_v_northward_0.gif']' returned non-zero exit status 2.

In [ ]:
#Check if the one for absolute val works and maybe rather do it as an if elif inside the first code????
#to save place, but dont do if it takes uncessary time 

def results_absolute_val_animation(file_path, variable1, variable2, dir, frame, start, **kwargs):
    ds = xr.open_dataset(file_path, engine="netcdf4")  #add .isel(s_rho = -1) when expanding to 3D model
    ds_var_1 = ds[f'{variable1}']
    ds_var_2 = ds[f'{variable2}']
    print(f'The following variables are selected: {variable1} and {variable2}')
    abs_val = np.sqrt((ds_var_1 **2) + (ds_var_2**2))
    longitude = ds["X"]
    latitude = ds["Y"]
    fig,ax = plt.subplots(figsize = (12,8), subplot_kw = {'projection' : ccrs.NorthPolarStereo()})
    sc = ax.pcolormesh(abs_val[start], cmap = cmocean.cm.speed, transform = ccrs.PlateCarree(), **kwargs)
    ax.add_feature(cartopy.feature.LAND, zorder = 1, edgecolor = 'black')
    cbar = fig.colorbar(sc, ax=ax, orientation = "vertical", label = '$\sqrt{{variable1}²+{variable2}²}$', extend = 'both')


    def update(frame):
        sc.set_array(abs_val[frame])
        ax.set_title(f'Time step: {frame*3} hrs')
        ax.set_xlabel(f'Longitude [$\circ$]')
        ax.set_ylabel(f'Latitude [$\circ$]')
        return sc 
    
    ani = FuncAnimation(fig, update, frames=range(frame), interval = 400)
    ani.save(f'{dir}/animation_abs_val_{variable1}_+_{variable2}.gif', writer="imagemagick")